We've been trying to use the analytical (Buckley-Leverett) solution of the two-phase flow in porous media to fit the Corey-type relative permeability model to the experimental oil recovery data. In this post, I'm going to compare the numerical solution of the same model with the analytical results. You can find the codes that I have written in [this github repository](https://github.com/simulkade/peteng). Here, I only call the codes and compare the results.

In [2]:
# load the input parameters and the functions
using Roots, PyPlot, Dierckx, JFVM
import JSON, JLD
include("rel_perms.jl")
include("forced_imbibition_corey.jl")
include("frac_flow_funcs.jl")
IJulia.clear_output()

0

The input parameters are stored in the `input_params_BL.jld` file, that can be loaded by

In [11]:
JLD.@load "results/input_params_BL.jld"

14-element Array{Symbol,1}:
 :L_core  
 :kro0    
 :krw0    
 :mu_oil  
 :mu_water
 :no      
 :nw      
 :perm_ave
 :poros   
 :pv_inj  
 :sor     
 :swc     
 :swi     
 :u_inj   

Now we can run the functions for the analytical and numerical solutions:

In [12]:
# call the functions
# numerical solution (finite volume)
(t_num, R_num, sw_prf)=forced_imb_impes(mu_water, mu_oil, u_inj, 
    poros, perm_ave, swc, sor, kro0, no,
    krw0,nw, swi, 1.0, L_core, pv_inj, Nx=50)

# Analytical solution (BL)
(xt_shock, sw_shock, xt_prf, sw_prf, t_anal, p_inj, R_anal) = frac_flow_wf(
  muw=mu_water, muo=mu_oil, ut=u_inj, phi=poros,
  k=perm_ave, swc=swc, sor=sor, kro0=kro0, no=no, 
  krw0=krw0, nw=nw, sw0=swi, sw_inj=1.0, L=L_core, pv_inj=pv_inj)
IJulia.clear_output() # only to clear the output from the previous function

progress is 0.0 %progress is 0.04902159180605293 %progress is 0.09804318361210586 %progress is 0.14754878842302446 %progress is 0.1999434585578175 %progress is 0.26029935386822073 %progress is 0.3362854229718309 %progress is 0.4147264505493329 %progress is 0.4931674781268349 %progress is 0.5730882851328756 %progress is 0.6718348489753654 %progress is 0.7587253158873429 %progress is 0.8456157827993206 %progress is 0.9372159594360676 %progress is 1.042687576042445 %progress is 1.1337826701199794 %progress is 1.2248777641975137 %progress is 1.3276546851800994 %progress is 1.4277299695585555 %progress is 1.5232716145841798 %progress is 1.6188132596098044 %progress is 1.7351876228180618 %progress is 1.8315336045470723 %progress is 1.9278795862760827 %progress is 2.030556084065056 %progress is 2.13997025810544 %progress is 2.237735235355564 %progress is 2.3355002126056887 %progress is 2.44967369435109 %progress is 2.5508589893818696 %progress is 2.6520442844126486 %progress is 2.754253471217

LoadError: ArgumentError: The interval [a,b] is not a bracketing interval.
You need f(a) and f(b) to have different signs (f(a) * f(b) < 0).
Consider a different bracket or try fzero(f, c) with an initial guess c.



Now, we can plot the results and compare the solutions:

In [14]:
plot(t_anal, R_anal, "o", t_num, R_num)
xlabel("time [s]")
ylabel("Recovery factor [-]")
IJulia.clear_output()

LoadError: UndefVarError: t_anal not defined

It seems that the match very well. But if we zoom on the recovery plot close to the water breakthrough time,

In [13]:
plot(t_anal, R_anal, "o", t_num, R_num)
xlabel("time [s]")
ylabel("Recovery factor [-]")
axis([25000, 50000, 0.40, 0.5])
IJulia.clear_output()

LoadError: UndefVarError: t_anal not defined

we can see that there is roughly 1 percent underestimation of the recovery factor by the numerical method. One reason is the numerical diffusion in the upwind scheme that I have used in my numerical solution. With this diffusion, the front is not sharp anymore and the water breakthrough (decrease in the slope of the recovery curve from the linear trend) happens a bit earlier in time. Let's test it by plotting the saturation profiles:

In [15]:
pv_inj2 = 0.3
(t_num, R_num, sw_prf_num)=forced_imb_impes(mu_water, mu_oil, u_inj, 
    poros, perm_ave, swc, sor, kro0, no,
    krw0,nw, swi, 1.0, L_core, pv_inj2, Nx=50)

# Analytical solution (BL)
(xt_shock, sw_shock, xt_prf, sw_prf, t_anal, p_inj, R_anal) = frac_flow_wf(
  muw=mu_water, muo=mu_oil, ut=u_inj, phi=poros,
  k=perm_ave, swc=swc, sor=sor, kro0=kro0, no=no, 
  krw0=krw0, nw=nw, sw0=swi, sw_inj=1.0, L=L_core, pv_inj=pv_inj2)
visualizeCells(sw_prf_num)
plot(xt_prf*t_anal[end], sw_prf)
axis([0, L_core, 0, 1.0])
legend(["Numerical", "Analytical"])
xlabel("Core length [m]")
ylabel("Water saturation [-]")
IJulia.clear_output() # only to clear the output from the previous function

progress is 0.0 %progress is 0.33334682428115986 %progress is 0.6666936485623197 %progress is 1.0033317612765662 %progress is 1.3596155181931588 %progress is 1.7700356063039009 %progress is 2.2867408762084493 %progress is 2.8201398637354633 %progress is 3.353538851262477 %progress is 3.897000338903553 %progress is 4.568476973032484 %progress is 5.159332148033932 %progress is 5.750187323035378 %progress is 6.37306852416526 %progress is 7.0902755170886245 %progress is 7.709722156815858 %progress is 8.329168796543092 %progress is 9.028051859224675 %progress is 9.708563792998175 %progress is 10.358246979172423 %progress is 11.007930165346668 %progress is 11.79927583516282 %progress is 12.45442851092009 %progress is 13.10958118667736 %progress is 13.807781371642378 %progress is 14.551797755116988 %progress is 15.216599600417835 %progress is 15.881401445718682 %progress is 16.657781121587412 %progress is 17.34584112779671 %progress is 18.033901134006005 %progress is 18.72892360427646 %progre

LoadError: ArgumentError: The interval [a,b] is not a bracketing interval.
You need f(a) and f(b) to have different signs (f(a) * f(b) < 0).
Consider a different bracket or try fzero(f, c) with an initial guess c.



We can clearly see that the extra numerical diffusion causes the water front to move faster resulting in an earlier water breakthrough. We can decrease this diffusion by refining the grid:

In [16]:
(t_num, R_num, sw_prf_num)=forced_imb_impes(mu_water, mu_oil, u_inj, 
    poros, perm_ave, swc, sor, kro0, no,
    krw0,nw, swi, 1.0, L_core, pv_inj2, Nx=500)
visualizeCells(sw_prf_num)
plot(xt_prf*t_anal[end], sw_prf)
axis([0, L_core, 0, 1.0])
legend(["Numerical", "Analytical"])
xlabel("Core length [m]")
ylabel("Water saturation [-]")
IJulia.clear_output()

progress is 0.0 %progress is 0.03333468242805048 %progress is 0.06666936485610096 %progress is 0.10033317612746036 %progress is 0.13596155181904163 %progress is 0.17700356063005812 %progress is 0.22867408762037472 %progress is 0.28201398637304326 %progress is 0.3353538851257118 %progress is 0.38970003388923014 %progress is 0.4568476973025844 %progress is 0.515933214802749 %progress is 0.5750187323029134 %progress is 0.6373068524159919 %progress is 0.7090275517085816 %progress is 0.7709722156814034 %progress is 0.8329168796542253 %progress is 0.902805185922532 %progress is 0.9708563792995408 %progress is 1.0358246979173524 %progress is 1.1007930165351645 %progress is 1.1799275835166405 %progress is 1.245442851092071 %progress is 1.3109581186675014 %progress is 1.3807781371636911 %progress is 1.4551797755111426 %progress is 1.5216599600408625 %progress is 1.5881401445705823 %progress is 1.665778112157749 %progress is 1.7345841127786157 %progress is 1.8033901133994827 %progress is 1.87289

progress is 17.08929486975696 %progress is 17.170798999556872 %progress is 17.24511679841036 %progress is 17.318785193100787 %progress is 17.392453587791213 %progress is 17.47764056583336 %progress is 17.548880065300523 %progress is 17.620119564767688 %progress is 17.701806712303295 %progress is 17.77603727527922 %progress is 17.84979630855748 %progress is 17.923555341835744 %progress is 18.00850033137906 %progress is 18.079777350321734 %progress is 18.151054369264415 %progress is 18.23293102016512 %progress is 18.30707037155842 %progress is 18.380923260545043 %progress is 18.454776149531664 %progress is 18.539469046842573 %progress is 18.610783694405427 %progress is 18.682098341968285 %progress is 18.76417240486272 %progress is 18.838216067098294 %progress is 18.9121668575044 %progress is 18.986117647910508 %progress is 19.070546682865825 %progress is 19.14189935501322 %progress is 19.213252027160614 %progress is 19.295533200078825 %progress is 19.36947603128473 %progress is 19.443418

progress is 34.714339395573305 %progress is 34.78722181870079 %progress is 34.86010424182828 %progress is 34.93570484911478 %progress is 35.01739801855459 %progress is 35.089354122341234 %progress is 35.16131022612788 %progress is 35.24646881152202 %progress is 35.31932327919824 %progress is 35.392177746874445 %progress is 35.46782456726593 %progress is 35.54944572878322 %progress is 35.62141697508115 %progress is 35.69338822137909 %progress is 35.778627520052794 %progress is 35.85145344824205 %progress is 35.9242793764313 %progress is 35.99997342089895 %progress is 36.08152090153693 %progress is 36.15350746309419 %progress is 36.225494024651454 %progress is 36.31081586349027 %progress is 36.38361257996804 %progress is 36.456409296445806 %progress is 36.53215174593718 %progress is 36.61362362202562 %progress is 36.68562572118441 %progress is 36.75762782034319 %progress is 36.84303432722142 %progress is 36.91580106291358 %progress is 36.98856779860576 %progress is 37.06436002474901 %pro

progress is 52.68081947571769 %progress is 52.75548251297728 %progress is 52.839437913527874 %progress is 52.911236982755014 %progress is 52.98303605198215 %progress is 53.06652293167975 %progress is 53.14035122996962 %progress is 53.214179528259486 %progress is 53.289013799163456 %progress is 53.37259314507955 %progress is 53.44443737951855 %progress is 53.51628161395754 %progress is 53.600071874146494 %progress is 53.67374286438932 %progress is 53.74741385463214 %progress is 53.82239910908238 %progress is 53.90565852857393 %progress is 53.97754258751395 %progress is 54.049426646453966 %progress is 54.13348374663468 %progress is 54.20702112829191 %progress is 54.28055850994913 %progress is 54.355679440703895 %progress is 54.43865994277566 %progress is 54.51057982011515 %progress is 54.582499697454644 %progress is 54.66679625250226 %progress is 54.74021739930329 %progress is 54.81363854610433 %progress is 54.88888352673905 %progress is 54.97161569539513 %progress is 55.04356837340214 %

progress is 70.90650536759622 %progress is 70.980269126778 %progress is 71.05403288595976 %progress is 71.12903438287533 %progress is 71.21242151065974 %progress is 71.28434719458957 %progress is 71.3562728785194 %progress is 71.44035049795751 %progress is 71.51395421216947 %progress is 71.58755792638144 %progress is 71.66271776394778 %progress is 71.745774262526 %progress is 71.81774101227441 %progress is 71.88970776202281 %progress is 71.97406542094235 %progress is 72.04753126042944 %progress is 72.12099709991652 %progress is 72.19630130328198 %progress is 72.27906530631762 %progress is 72.35106948969747 %progress is 72.42307367307734 %progress is 72.50768639574306 %progress is 72.58103056643004 %progress is 72.65437473711702 %progress is 72.72981298852076 %progress is 72.81231261779682 %progress is 72.88435158784964 %progress is 72.95639055790247 %progress is 73.04123999100021 %progress is 73.11447450533363 %progress is 73.18770901966704 %progress is 73.2632738639165 %progress is 73

progress is 89.20536209135385 %progress is 89.29057105374648 %progress is 89.36368364978236 %progress is 89.43679624581824 %progress is 89.51257145109167 %progress is 89.59451193670245 %progress is 89.6666583410992 %progress is 89.73880474549594 %progress is 89.8242458918541 %progress is 89.89725740161144 %progress is 89.97026891136878 %progress is 90.04617306159172 %progress is 90.127877897438 %progress is 90.20005776424449 %progress is 90.27223763105098 %progress is 90.35790676058558 %progress is 90.43082176469247 %progress is 90.50373676879936 %progress is 90.57976953530923 %progress is 90.66124460635554 %progress is 90.73345797865667 %progress is 90.8056713509578 %progress is 90.89156801598826 %progress is 90.96438938255976 %progress is 91.03721074913128 %progress is 91.11337387078451 %progress is 91.19462125407759 %progress is 91.26686873971728 %progress is 91.33911622535697 %progress is 91.42524368450388 %progress is 91.49797276841977 %progress is 91.57070185233567 %progress is 9

LoadError: UndefVarError: xt_prf not defined

Now, we can see that the numerical solution is very close to the analytical solution. This must give a better match for the recovery curves as well:

In [9]:
# numerical solution (finite volume)
(t_num, R_num, sw_prf)=forced_imb_impes(mu_water, mu_oil, u_inj, 
    poros, perm_ave, swc, sor, kro0, no,
    krw0,nw, swi, 1.0, L_core, pv_inj, Nx=500)

# Analytical solution (BL)
(xt_shock, sw_shock, xt_prf, sw_prf, t_anal, p_inj, R_anal) = frac_flow_wf(
  muw=mu_water, muo=mu_oil, ut=u_inj, phi=poros,
  k=perm_ave, swc=swc, sor=sor, kro0=kro0, no=no, 
  krw0=krw0, nw=nw, sw0=swi, sw_inj=1.0, L=L_core, pv_inj=pv_inj)
plot(t_anal, R_anal, "o", t_num, R_num)
xlabel("time [s]")
ylabel("Recovery factor [-]")
axis([25000, 50000, 0.40, 0.5])
IJulia.clear_output()

LoadError: UndefVarError: mu_water not defined

## sensitivity analysis
Effect of relperm parameters on the total recovery factor, compare to a base case:

In [ ]:
(t_num, R_num, sw_prf)=forced_imb_impes(mu_water, mu_oil, u_inj, 
    poros, perm_ave, swc, sor, kro0, no,
    krw0,nw, swi, 1.0, L_core, pv_inj, Nx=500)

# Analytical solution (BL)
(xt_shock, sw_shock, xt_prf, sw_prf, t_anal, p_inj, R_anal) = frac_flow_wf(
  muw=mu_water, muo=mu_oil, ut=u_inj, phi=poros,
  k=perm_ave, swc=swc, sor=sor, kro0=kro0, no=no, 
  krw0=krw0, nw=nw, sw0=swi, sw_inj=1.0, L=L_core, pv_inj=pv_inj)
plot(t_anal, R_anal, "o", t_num, R_num)
xlabel("time [s]")
ylabel("Recovery factor [-]")
axis([25000, 50000, 0.40, 0.5])
IJulia.clear_output()

In [1]:
kro0

LoadError: UndefVarError: kro0 not defined